In [ ]:
!pip install importlib-metadata

In [ ]:
from cvrp import CVRP
from state import State
from action import Action
from utils import transition, cost

In [ ]:
cities = [[0, 0], [1, 2], [3, 5], [6, 8]]  # Depot at [0,0], and three cities
demands = [0, 2, 3, 1]  # Depot demand is 0, other cities have demand
capacity = 5

In [ ]:
cvrp_instance = CVRP(cities=cities, demands=demands, capacity=capacity)
initial_state = State(cvrp_instance)

In [ ]:
cvrp_instance.distance_matrix

In [ ]:
action_generator = Action(cvrp_instance, initial_state)

In [ ]:
feasible_actions = action_generator.get_all_actions()
print("\nFeasible Actions from Initial State:")
for route in feasible_actions:
    print(route)

In [ ]:
example_action = feasible_actions[0]  # Pick the first feasible action
new_state = transition(initial_state, example_action)

In [ ]:
if new_state:
    print("\nNew State (Visited Status):", new_state.visited)
    route_cost = cost(cvrp_instance, example_action)
    print(f"Cost of route {example_action}: {route_cost}")
else:
    print("The chosen action revisits cities and is therefore invalid.")

In [ ]:
new_action_generator = Action(cvrp_instance, new_state)
feasible_actions = new_action_generator.get_all_actions()
print("\nFeasible Actions from new State:")
for route in feasible_actions:
    print(route)

In [ ]:
example_action = feasible_actions[2]
new_state = transition(new_state, example_action)
if new_state:
    print("\nNew State (Visited Status):", new_state.visited)
    route_cost = cost(cvrp_instance, example_action)
    print(f"Cost of route {example_action}: {route_cost}")
else:
    print("The chosen action revisits cities and is therefore invalid.")

In [ ]:
from policy import policy_evaluation_with_milp

## Test 2

In [ ]:
cities = [(2, 3), (5, 8), (1, 9), (7, 3)]
demands = [0, 2, 3, 4, 5]
capacity = 10
cvrp_instance = CVRP(cities=[(0, 0)] + cities, demands=demands, capacity=capacity)


In [ ]:
initial_state = State(cvrp_instance)
print("Initial State (visited):", initial_state.visited)


In [ ]:
action_generator = Action(cvrp_instance, initial_state)
feasible_actions = action_generator.get_all_actions()
print("Feasible Actions:", feasible_actions)

In [ ]:
from policy import ValueNetwork
from utils import solve_milp_with_value_function

In [ ]:
value_network = ValueNetwork(input_size=len(initial_state.encode_state()), hidden_dim=16)
route = solve_milp_with_value_function(cvrp_instance, initial_state, value_network)
print("MILP Optimal Route:", route)

## Testing OR-tools


In [1]:
from cvrp import CVRP
from ORtools import ORToolsSolver
from parser import parse_cvrplib

In [2]:
file_path = "C:/Users/issam/OneDrive/Desktop/phd/Vrp-Set-A/A/A-n32-k5.vrp"
depot, cities, demands, capacity, num_vehicles = parse_cvrplib(file_path)

In [3]:
cvrp_instance = CVRP(cities=[cities[depot]] + cities, demands=[0] + demands, capacity=capacity, num_vehicles=num_vehicles ,depot_index=0)

In [4]:
ortools_solver = ORToolsSolver(cvrp_instance)
routes, total_cost = ortools_solver.solve()

Starting solve...
Solution found: True
Vehicle 0 route cost: 38.8329756778952
Vehicle 1 route cost: 448.7855540317404
Vehicle 2 route cost: 418.44835371064516
Vehicle 3 route cost: 490.8149633906056
Vehicle 4 route cost: 660.9976624448711
Routes: [[0, 2, 0], [0, 10, 8, 7, 6, 5, 4, 3, 0], [0, 17, 16, 14, 13, 12, 9, 0], [0, 25, 22, 21, 20, 18, 15, 11, 0], [0, 32, 31, 30, 29, 28, 27, 26, 24, 23, 19, 1, 0]]
Cost: 2057.8795092557575


In [ ]:
total_cost 

In [5]:
print("\nRoutes:")
for i, route in enumerate(routes):
    route_demand = sum(cvrp_instance.demands[city] for city in route)
    print(f"Vehicle {i}: {route} (demand: {route_demand}/{capacity})")


Routes:
Vehicle 0: [0, 2, 0] (demand: 19/100)
Vehicle 1: [0, 10, 8, 7, 6, 5, 4, 3, 0] (demand: 97/100)
Vehicle 2: [0, 17, 16, 14, 13, 12, 9, 0] (demand: 97/100)
Vehicle 3: [0, 25, 22, 21, 20, 18, 15, 11, 0] (demand: 98/100)
Vehicle 4: [0, 32, 31, 30, 29, 28, 27, 26, 24, 23, 19, 1, 0] (demand: 99/100)
